In [ ]:
import scanpy as sc
import scipy.sparse as sp

# Read h5ad file
print("Reading h5ad file...")
adata = sc.read_h5ad('../scRNA_Project/processed_data/strict_epdsc_annotated_data.h5')

print("Converting X to CSC sparse matrix...")
# Convert main matrix to CSC (fast for gene queries)
if sp.issparse(adata.X):
    adata.X = adata.X.tocsc()

print("Converting layers to CSC sparse matrices...")
# Convert all layers to CSC if they are sparse
for layer in adata.layers:
    if sp.issparse(adata.layers[layer]):
        adata.layers[layer] = adata.layers[layer].tocsc()
    else:
        adata.layers[layer] = sp.csc_matrix(adata.layers[layer])

print("Creating Zarr store...")
# Save as Zarr
adata.write_zarr('../scRNA_Project/processed_data/strict_epdsc_annotated_data_full_csc.zarr')


Reading h5ad file...
Converting X to CSC sparse matrix...
Converting layers to CSC sparse matrices...
Creating Zarr store...


In [ ]:
import scanpy as sc
import anndata as ad

# Read h5ad file
print("Reading h5ad file...")
adata = ad.read_zarr('https://rubioinformatics.s3.us-east-1.amazonaws.com/Test_zarr/corrected_doublet_decontX_mt_Duo_allergy.zarr')

sc.tl.rank_genes_groups(adata, 'cyclon_Phase', method='wilcoxon', groups=['G1', 'S', 'G2M'])

NameError: name 'use_condaenv' is not defined

In [10]:
import zarr

# Open the Zarr file as a group (read-only mode by default)
root = zarr.open('https://scrnaseq-browser.s3.us-east-2.amazonaws.com/strict_epdsc_annotated_data_csc_full.zarr/', mode='r')

# List top-level keys to inspect the structure (e.g., ['X', 'obs', 'var', 'layers', ...])
print(root.info)  # Prints a tree-like summary of the hierarchy
print(list(root.keys()))  # Or just list the keys

Name        : 
Type        : Group
Zarr format : 2
Read-only   : True
Store type  : FsspecStore
No. members : 250
No. arrays  : 193
No. groups  : 57
['var', 'X', 'layers', 'varp', 'raw', 'obs', 'varm', 'uns', 'obsp', 'obsm']


In [4]:
import zarr
import numpy as np
import json
import s3fs
import os

# --- 1. AWS Configuration and Credentials ---

# Define the S3 path for the Zarr store
S3_URI = 's3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr'

# Load credentials from JSON (as confirmed to be working)
try:
    with open('./credentials.json', 'r') as f:
        creds = json.load(f)['default']
        aws_access_key_id = creds['aws_access_key_id']
        aws_secret_access_key = creds['aws_secret_access_key']
except FileNotFoundError:
    print("Error: credentials.json not found. Please ensure it is in the correct directory.")
    exit()
except KeyError:
    print("Error: 'default' or key/secret access key missing from credentials.json.")
    exit()

# Create authenticated S3 filesystem
fs = s3fs.S3FileSystem(
    anon=False,
    key=aws_access_key_id,
    secret=aws_secret_access_key
)

# Open Zarr store using s3fs for authenticated read/write access
store = s3fs.S3Map(root=S3_URI, s3=fs, check=False)

# --- 2. Data Filtering Logic ---

# Open the store in read mode ('r') for the initial data access
# We will reopen in 'a' mode later for the write operation
print(f"Opening Zarr store: {S3_URI}...")
root_r = zarr.open(store, mode='r')

# Access 'obs/celltype' components
try:
    celltype_group = root_r['obs/celltype']
    codes = celltype_group['codes'][:]
    categories = celltype_group['categories'][:]

    # Decode categories from bytes to strings if necessary
    if categories.dtype.kind == 'S':
        categories = np.array([x.decode('utf-8') for x in categories])

except KeyError as e:
    print(f"Error accessing required data groups: {e}")
    exit()


# Choose filter and find matching index
target_celltype = 'DB_stress'
print(f"Filtering for target cell type: '{target_celltype}'")

category_idx = np.where(categories == target_celltype)[0]

if len(category_idx) == 0:
    print("Available cell types:", categories)
    raise ValueError(f"Cell type '{target_celltype}' not found in categories. Please choose an available type.")

category_idx = category_idx[0]
filter_mask = codes == category_idx
passing_indices = np.where(filter_mask)[0]

n_filtered_cells = len(passing_indices)
print(f"Found {n_filtered_cells} cells passing the filter.")

# --- 3. Write Filter Results to Metadata ---

# Reopen the store in append mode ('a') for writing metadata
print("\nOpening store in write mode ('a') to update metadata...")
root_a = zarr.open(store, mode='a')

# Write metadata to the Zarr root group (.zattrs)
root_a.attrs['last_filter_timestamp'] = str(np.datetime64('now'))
root_a.attrs['filtered_cell_indices'] = passing_indices.tolist()
root_a.attrs['filter_criterion'] = f"Cells with celltype == '{target_celltype}'"
root_a.attrs['n_filtered_cells'] = n_filtered_cells
print("Wrote metadata to Zarr root attributes.")

# Write metadata to the 'obs' group attributes (optional, but useful)
obs_group = root_a['obs']
obs_group.attrs['last_filter_timestamp'] = str(np.datetime64('now'))
obs_group.attrs['filtered_cell_indices'] = passing_indices.tolist()
obs_group.attrs['filter_criterion'] = f"Cells with celltype == '{target_celltype}'"
print("Wrote metadata to 'obs' group attributes.")


# Consolidate metadata (ensures all metadata updates are written efficiently)
print("\nConsolidating metadata (This is the time-consuming S3 write step)...")
zarr.consolidate_metadata(store)
print("Consolidation complete. Metadata successfully written. 🎉")

# Verification (optional)
print("\n--- Verification (Reading back new attributes) ---")
root_check = zarr.open(store, mode='r')
print("Root metadata filter criterion:", root_check.attrs.get('filter_criterion'))
print("Obs metadata filter criterion:", root_check['obs'].attrs.get('filter_criterion'))


Opening Zarr store: s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr...
Filtering for target cell type: 'DB_stress'
Found 1192 cells passing the filter.

Opening store in write mode ('a') to update metadata...
Wrote metadata to Zarr root attributes.
Wrote metadata to 'obs' group attributes.

Consolidating metadata (This is the time-consuming S3 write step)...
Consolidation complete. Metadata successfully written. 🎉

--- Verification (Reading back new attributes) ---
Root metadata filter criterion: Cells with celltype == 'DB_stress'
Obs metadata filter criterion: Cells with celltype == 'DB_stress'


In [5]:
import boto3
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from botocore.exceptions import ClientError

# S3 bucket and folder details
bucket_name = 'scrnaseq-browser'
# zarr_folder = './processed_data/strict_epdsc_annotated_data_csc_full.zarr'
zarr_folder = './processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/'
s3_prefix = 'corrected_doublet_decontX_mt_Duo_allergy.zarr'  # S3 folder path

# Initialize S3 client
s3_client = boto3.client('s3')

# Function to upload a single file to S3
def upload_file(local_file, bucket, s3_key):
    try:
        s3_client.upload_file(local_file, bucket, s3_key)
        print(f'Successfully uploaded {local_file} to s3://{bucket}/{s3_key}')
        return True
    except ClientError as e:
        print(f'Error uploading {local_file}: {e}')
        return False

# Function to upload zarr folder in parallel
def upload_zarr_to_s3(local_path, bucket, s3_path, max_workers=10):
    # Collect all files to upload
    upload_tasks = []
    for root, _, files in os.walk(local_path):
        for file in files:
            local_file = os.path.join(root, file)
            s3_key = os.path.join(s3_path, os.path.relpath(local_file, local_path)).replace('\\', '/')
            upload_tasks.append((local_file, bucket, s3_key))

    # Upload files in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(upload_file, lf, b, sk): lf for lf, b, sk in upload_tasks}
        for future in as_completed(future_to_file):
            local_file = future_to_file[future]
            if not future.result():
                print(f'Failed to upload {local_file}')

# Upload the zarr folder
try:
    upload_zarr_to_s3(zarr_folder, bucket_name, s3_prefix)
    print(f"Successfully uploaded {zarr_folder} to s3://{bucket_name}/{s3_prefix}")
except Exception as e:
    print(f"Error during upload: {e}")
    exit(1)

Successfully uploaded ./processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/highly_variable/.zarray to s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/highly_variable/.zarray
Successfully uploaded ./processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/.zgroup to s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/.zgroup
Successfully uploaded ./processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/.zattrs to s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr/.zattrs
Successfully uploaded ./processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/highly_variable/.zattrs to s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/highly_variable/.zattrs
Successfully uploaded ./processed_data/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/.zattrs to s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr/var/.zattrs
Successfully uploaded ./processed_data/corre

In [10]:
import requests
import numpy as np
import pandas as pd
import json
import fsspec
import zarr

PROXY_URL = "http://localhost:8080/add_version"
LIST_URL = "http://localhost:8080/list_versions"

# Generate sample data
def generate_sample_data():
    cell_mask = np.array([True, False, True, True, False])
    umap_coords = np.random.rand(5, 2)
    clusters = np.array([0, 1, 0, 2, 1])
    qc_df = pd.DataFrame({
        'n_genes': [500, 300, 600, 400, 200],
        'n_counts': [10000, 8000, 12000, 9000, 5000],
        'percent_mito': [2.1, 1.8, 2.5, 1.9, 3.2]
    })
    markers_df = pd.DataFrame({
        'gene': ['GeneA', 'GeneB', 'GeneC'],
        'logFC': [1.5, -0.8, 2.1],
        'p_value': [0.01, 0.03, 0.001],
        'cluster': [0, 1, 0]
    })
    return {
        'cell_mask': cell_mask,
        'umap_coords': umap_coords,
        'clusters': clusters,
        'qc_summary': qc_df,
        'markers': markers_df
    }

# Verify version contents
def verify_version(zarr_url, version_prefix, credential_id='default'):
    try:
        fs = fsspec.filesystem('s3', anon=True)
        version_path = f"{zarr_url}/versions/{version_prefix}"
        
        # Check .zattrs
        zattrs_path = f"{version_path}/.zattrs"
        metadata = None
        if fs.exists(zattrs_path):
            with fs.open(zattrs_path, 'r') as f:
                metadata = json.load(f)
        else:
            print(f"Warning: .zattrs not found at {zattrs_path}")
        
        # Check Parquet files
        parquet_files = fs.glob(f"{version_path}/results/*.parquet")
        parquet_data = {}
        for p in parquet_files:
            key = p.split('/')[-1].replace('.parquet', '')
            print(f"Reading Parquet: {p}")
            with fs.open(p, 'rb') as f:
                df = pd.read_parquet(f)
            parquet_data[key] = df.to_dict('records')
        
        # Check Zarr arrays (exclude .parquet files)
        zarr_arrays = {}
        array_paths = fs.glob(f"{version_path}/masks/*") + fs.glob(f"{version_path}/results/*/*")
        for array_path in array_paths:
            if array_path.endswith('.parquet'):
                continue  # Skip Parquet files
            array_name = array_path.split(version_path + '/')[-1]
            print(f"Reading Zarr array: {array_path}")
            store = fs.get_mapper(array_path)
            try:
                z = zarr.open(store, mode='r')
                zarr_arrays[array_name] = z[:].tolist()
            except Exception as e:
                print(f"Error reading Zarr array {array_name}: {str(e)}")
        
        return {
            'metadata': metadata,
            'parquet_data': parquet_data,
            'zarr_arrays': zarr_arrays
        }
    except Exception as e:
        return f"Error verifying version: {str(e)}"

test_cases = [
    {
        "zarr_url": "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr",
        "user_id": "user123",
        "version_prefix": None,
        "metadata": {
            "filter_criterion": "Cells with n_genes > 200",
            "n_filtered_cells": 3,
            "analysis_date": "2025-10-26"
        }
    },
    {
        "zarr_url": "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr",
        "user_id": "user123",
        "version_prefix": "user123_v1",
        "metadata": {
            "filter_criterion": "Cells with n_genes > 300",
            "n_filtered_cells": 2,
            "analysis_date": "2025-10-26",
            "method": "Seurat v4"
        }
    }
]

def main():
    print("Testing Zarr Versioning System...\n")
    
    sample_data = generate_sample_data()
    
    for i, test in enumerate(test_cases, 1):
        print(f"--- Test Case {i}: {test['version_prefix'] or 'Auto-generated'} ---")
        
        payload = {
            "zarr_url": test["zarr_url"],
            "user_id": test["user_id"],
            "version_prefix": test["version_prefix"],
            "metadata": test["metadata"],
            "parquet_data": {
                "qc_summary": sample_data['qc_summary'].to_dict('records'),
                "markers": sample_data['markers'].to_dict('records')
            },
            "zarr_arrays": {
                "cell_mask": sample_data['cell_mask'].tolist(),
                "umap/coords": sample_data['umap_coords'].tolist(),
                "clusters": sample_data['clusters'].tolist()
            },
            "credential_id": "default"
        }
        
        try:
            response = requests.post(PROXY_URL, json=payload)
            response.raise_for_status()
            result = response.json()
            print(f"Response: {json.dumps(result, indent=2)}")
            
            version_prefix = result.get('version_prefix', test['version_prefix'])
            verification = verify_version(test['zarr_url'], version_prefix)
            print(f"Verified Version Contents: {json.dumps(verification, indent=2)}")
        except requests.RequestException as e:
            print(f"Request Error: {str(e)}")
        print()
    
    print("--- All Versions ---")
    try:
        response = requests.get(LIST_URL, params={"zarr_url": "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr"})
        response.raise_for_status()
        print(json.dumps(response.json(), indent=2))
    except requests.RequestException as e:
        print(f"Error listing versions: {str(e)}")

if __name__ == "__main__":
    main()

Testing Zarr Versioning System...

--- Test Case 1: Auto-generated ---
Response: {
  "metadata_keys": [
    "filter_criterion",
    "n_filtered_cells",
    "analysis_date"
  ],
  "status": "Version created",
  "user_id": "user123",
  "version_path": "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions/user123_20251026_193819",
  "version_prefix": "user123_20251026_193819",
  "zarr_url": "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr"
}
Reading Parquet: scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions/user123_20251026_193819/results/markers.parquet
Reading Parquet: scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions/user123_20251026_193819/results/qc_summary.parquet
Reading Zarr array: scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions/user123_20251026_193819/masks/cell_mask
Reading Zarr array: scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions/user123_20251026_193819/masks/cluste

In [12]:
import fsspec
import json
import os
import logging

# --- Configuration (Must match app.py) ---
ZARR_URL = "s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr"
CREDENTIAL_ID = 'default' # Or whichever ID you are using

logging.basicConfig(level=logging.INFO)

# --- Functions from app.py ---

# Modified load_credentials function for robustness

def load_credentials():
    # Fallback for interactive execution where __file__ is undefined
    try:
        # Standard way (works when running as a script)
        base_dir = os.path.dirname(__file__)
    except NameError:
        # Fallback (assumes the script is being run from its directory)
        base_dir = os.getcwd() 
        
    cred_file = os.path.join(base_dir, 'credentials.json')
    
    if not os.path.exists(cred_file):
        # You can try a hardcoded path for the environment here if needed
        raise FileNotFoundError(f"credentials.json not found at {cred_file}")
        
    with open(cred_file, 'r') as f:
        return json.load(f)

CREDENTIALS = load_credentials()

def get_filesystem(credential_id='default'):
    creds = CREDENTIALS.get(credential_id)
    if not creds:
        raise ValueError(f"Credential ID {credential_id} not found")
    
    # We explicitly include the region as a guess, as S3 URLs often omit it
    # Use 'us-east-2' if your bucket is there, or omit 'client_kwargs' if your default AWS config is set up
    # Note: fsspec often deduces region, but explicit can help debug.
    
    # The crucial part: using the loaded credentials
    return fsspec.filesystem('s3', 
                             key=creds.get('aws_access_key_id'), 
                             secret=creds.get('aws_secret_access_key'))

# --- S3 Verification Logic ---

def test_s3_connection(zarr_url):
    logging.info(f"Attempting connection test for: {zarr_url}")
    
    try:
        fs = get_filesystem(CREDENTIAL_ID)
        
        # 1. Define the versions directory path
        versions_dir = f"{zarr_url}/versions"
        logging.info(f"Checking directory: {versions_dir}")
        
        # 2. Test existence (This is where fsspec/boto3 fails)
        if not fs.exists(versions_dir):
            logging.warning(f"Versions directory not found: {versions_dir}")
            return {'status': 'success', 'message': 'Directory checked, no versions found.'}

        # 3. Test listing the contents (This is what /list_versions does)
        version_folders = [f['name'].split('/')[-1] for f in fs.ls(versions_dir, detail=True) if f['type'] == 'directory']
        
        logging.info(f"Successfully found version folders: {version_folders}")
        return {'status': 'success', 'versions_found': len(version_folders)}

    except Exception as e:
        logging.error(f"--- FAILED S3 CONNECTION TEST ---")
        logging.error(f"Error Type: {type(e).__name__}")
        logging.error(f"Error Message: {str(e)}")
        logging.error(f"---------------------------------")
        return {'status': 'error', 'message': str(e)}

# --- Run Test ---
if __name__ == "__main__":
    result = test_s3_connection(ZARR_URL)
    print("\n--- Test Result ---")
    print(json.dumps(result, indent=2))

INFO:root:Attempting connection test for: s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr
INFO:root:Checking directory: s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr/versions
INFO:root:Successfully found version folders: ['goguxor_20251027_000318']



--- Test Result ---
{
  "status": "success",
  "versions_found": 1
}


In [15]:
import json
import fsspec

# --- Configuration ---
# 1. Base URL
# The Python fsspec open function handles the HTTPS URL format directly.
zarr_url_base = "https://scrnaseq-browser.s3.us-east-2.amazonaws.com/strict_epdsc_annotated_data_csc_full.zarr"

# 2. Dynamic Variables
# You need to ensure this prefix exists and that the file is publicly readable.
version_prefix = "goguxor_20251027_051741" 

# --- Execution ---

try:
    # 3. Construct the full path to the specific metadata file (.zattrs)
    zattrs_url = f"{zarr_url_base}/versions/{version_prefix}/.zattrs"
    
    # 4. Use fsspec.open to get a file-like object for the HTTPS URL
    # mode='r' is the default for text mode reading.
    # The 'anon=True' or similar options aren't strictly needed for the 'https'
    # protocol implementation but are passed through if needed.
    # We use fsspec.open for simplicity, which works similarly to fs.open.
    with fsspec.open(zattrs_url, mode='r') as f:
        
        # Check the size first to see if the file is truly empty
        # Note: HTTP/S implementation might not always provide .size reliably
        # info = fsspec.info(zattrs_url)
        # print(f"File size: {info['size']} bytes") 
        
        try:
            # 5. Read and parse the JSON content
            metadata = json.load(f)
            
            # 6. Check for empty metadata after parsing
            if metadata:
                print(f"✅ Metadata loaded successfully for version '{version_prefix}'.")
                print(f"Keys: {list(metadata.keys())}")
                # print(f"Full Metadata: {metadata}") 
            else:
                # This executes if the file contains only "{}", which is technically valid JSON.
                print(f"⚠️ Warning: .zattrs found but contains empty metadata ({{}}) at {zattrs_url}")

        except json.JSONDecodeError as jde:
            print(f"❌ Error: Found file, but failed to parse JSON. Content may be corrupted.")
            # Optional: Read raw content to inspect corruption
            # f.seek(0); print(f"Raw content: {f.read()[:100]}...") 
            metadata = None

except FileNotFoundError:
    # This catches 404 Not Found errors from the HTTP request
    print(f"❌ Error: File Not Found at {zattrs_url}")
except Exception as e:
    print(f"❌ An unexpected error occurred during access: {e}")
    metadata = None

✅ Metadata loaded successfully for version 'goguxor_20251027_051741'.
Keys: ['filter_criterion', 'n_filtered_cells', 'analysis_date']


In [16]:
import scanpy as sc
import anndata
import dask.array as da
import zarr
from anndata.experimental import read_dispatched, read_elem
import numpy as np

def read_dask(store):
    f = zarr.open(store, mode="r")

    def callback(func, elem_name: str, elem, iospec):
        if iospec.encoding_type in ("dataframe", "csr_matrix", "csc_matrix", "awkward-array"):
            return read_elem(elem)  # Load metadata eagerly
        elif iospec.encoding_type == "array":
            return da.from_zarr(elem)  # Load arrays as Dask arrays (lazy)
        else:
            return func(elem)

    adata = read_dispatched(f, callback=callback)
    return adata

# Load the remote Zarr store
zarr_url = "https://scrnaseq-browser.s3.us-east-2.amazonaws.com/strict_epdsc_annotated_data_csc_full.zarr"
adata_dask = read_dask(zarr_url)

# Filter obs to include only the two groups
mask = adata_dask.obs['celltype'].isin(['DB_advanced', 'DB_stress'])
subset_indices = np.where(mask)[0]  # Get indices of cells in the two groups

# Subset the AnnData object
adata_subset = anndata.AnnData(
    X=adata_dask.X[subset_indices, :],  # Dask array slice, lazy-loaded
    obs=adata_dask.obs.iloc[subset_indices, :],  # Subset obs metadata
    var=adata_dask.var,  # Keep all var metadata
    uns=adata_dask.uns,  # Copy unstructured data
    obsm={k: adata_dask.obsm[k][subset_indices, :] for k in adata_dask.obsm.keys()},  # Subset embeddings
    layers={k: adata_dask.layers[k][subset_indices, :] for k in adata_dask.layers.keys()}  # Subset layers
)

/home/genos/gmunoz/SCRATCH/miniconda3/envs/sc_rna_env_python2/lib/python3.11/site-packages/anndata/experimental/__init__.py:48: FutureWarning: Importing read_elem from `anndata.experimental` is deprecated. Import anndata.io.read_elem instead.
  return module_get_attr_redirect(


In [32]:
import zarr
import time

zarr_url = "https://scrnaseq-browser.s3.us-east-2.amazonaws.com/strict_epdsc_annotated_data_csc_full.zarr"
f = zarr.open(zarr_url, mode="r")
print(f.tree())  # Print the hierarchy
if "X" in f:
    print(f["X"].info)  # Check X's details
else:
    print("X is not in the root group. Checking subgroups...")
    for key in f:
        print(f"{key}: {f[key].info}")

ImportError: 'rich' is required for Group.tree

In [30]:
subset_indices

array([   0,    1,    2, ..., 7687, 7688, 7691], shape=(1607,))

In [22]:
adata = anndata.read_zarr(zarr_url)

In [20]:
# Run differential expression analysis
sc.tl.rank_genes_groups(
    adata_subset,
    groupby='celltype',
    groups=['DB_advanced', 'DB_stress'],
    method='wilcoxon',  # Or another method like 't-test'
    use_raw=False  # Use adata_subset.X (ensure it contains the correct data, e.g., normalized counts)
)

# Compute the results (this triggers Dask to fetch the necessary data)
adata_subset.uns['rank_genes_groups']

{'params': {'groupby': 'celltype',
  'reference': 'rest',
  'method': 'wilcoxon',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'names': rec.array([('DSP', 'FTH1'), ('SLCO3A1', 'APOE'), ('SLC24A3', 'KRT14'),
            ..., ('KRT14', 'SLC24A3'), ('APOE', 'SLCO3A1'),
            ('FTH1', 'DSP')],
           dtype=[('DB_advanced', 'O'), ('DB_stress', 'O')]),
 'scores': rec.array([( 28.89136 ,  28.63564 ), ( 28.479958,  26.662031),
            ( 28.361862,  26.145124), ..., (-26.145124, -28.361862),
            (-26.662031, -28.479958), (-28.63564 , -28.89136 )],
           dtype=[('DB_advanced', '<f4'), ('DB_stress', '<f4')]),
 'pvals': rec.array([(1.53298342e-183, 2.41997276e-180),
            (2.07489002e-178, 1.29807778e-156),
            (5.97665032e-177, 1.11974486e-150), ...,
            (1.11974486e-150, 5.97665032e-177),
            (1.29807778e-156, 2.07489002e-178),
            (2.41997276e-180, 1.53298342e-183)],
           dtype=[('DB_advanced

In [35]:
"""
Fast DGE from Zarr on S3 using Scanpy

Ultra-optimized gene ranking between two groups from Zarr-stored AnnData.
Only loads cells from specified groups - never touches the full dataset.

Key features:
- Reads only the groupby column from obs (skip metadata)
- Loads sparse X matrix and converts to dense for computation
- Uses t-test_overestim_var (3-4x faster than wilcoxon)
- Minimal preprocessing (normalize_total + log1p only)
- Detailed timing breakdown
- Works with both AnnData zarr formats

Speed: ~4-7s depending on dataset size (S3 network I/O bound)
"""

import scanpy as sc
import anndata as ad
import zarr
import numpy as np
import pandas as pd
import time
from scipy import sparse


def rank_genes_zarr(
    s3_path: str,
    groupby: str,
    groups: list,
    method: str = "t-test_overestim_var",
    n_genes: int = None,
    verbose: bool = True
):
    """
    Fast differential gene expression from Zarr on S3.
    
    Only loads cells from the specified groups - never loads the full dataset.
    Uses optimized sparse matrix operations and fast statistical testing.
    
    Parameters
    ----------
    s3_path : str
        S3 path to zarr file (e.g., "s3://bucket/file.zarr")
    groupby : str
        Column in obs for grouping (e.g., "celltype", "Phase")
    groups : list
        Two groups to compare (e.g., ["G1", "S"])
    method : str, default "t-test_overestim_var"
        Statistical test to use. Options:
        - "t-test_overestim_var" (FASTEST, ~1-2s)
        - "t-test" (fast, ~1-3s)
        - "wilcoxon" (slower, ~2-5s, most robust)
    n_genes : int, optional
        Only return top N genes. If None, returns all genes.
        Setting to 500-1000 can speed up ranking by 2-3x.
    verbose : bool, default True
        Print timing breakdown
    
    Returns
    -------
    pd.DataFrame
        Results dataframe with columns: names, scores, logfoldchanges, pvals, pvals_adj
        Sorted by score (highest first)
    
    Examples
    --------
    >>> result = rank_genes_zarr(
    ...     s3_path="s3://bucket/data.zarr",
    ...     groupby="Phase",
    ...     groups=["G1", "S"],
    ...     method="t-test_overestim_var",
    ...     n_genes=500
    ... )
    >>> print(result.head(10))
    """
    
    t_total_start = time.time()
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"🚀 Fast Zarr DGE: {s3_path.split('/')[-1]}")
        print(f"   {groupby}: {groups[0]} vs {groups[1]}")
        print(f"{'='*70}")
    
    # ========== OPEN ZARR ==========
    t = time.time()
    f = zarr.open(s3_path, mode="r")
    t_open = time.time() - t
    
    # ========== READ GROUPBY COLUMN ONLY ==========
    t = time.time()
    obs_group = f["obs"]
    obs_keys = list(obs_group.keys())
    
    # Determine n_obs
    index_key = "_index" if "_index" in obs_keys else None
    if index_key is None:
        for key in obs_keys:
            item = obs_group[key]
            if isinstance(item, zarr.Group) and "codes" in item:
                n_obs = item["codes"].shape[0]
                break
    else:
        n_obs = obs_group[index_key].shape[0]
    
    # Read groupby column
    if groupby not in obs_keys:
        raise ValueError(f"Column '{groupby}' not in obs. Available: {list(obs_keys)}")
    
    groupby_item = obs_group[groupby]
    if isinstance(groupby_item, zarr.Group) and "categories" in groupby_item:
        categories = np.array(groupby_item["categories"][:], dtype=object)
        codes = np.array(groupby_item["codes"][:], dtype=int)
        groupby_col = categories[codes]
    else:
        groupby_col = np.array(groupby_item[:], dtype=object)
    
    t_obs = time.time() - t
    
    # ========== FIND CELLS FOR EACH GROUP ==========
    t = time.time()
    group_indices = {}
    for g in groups:
        mask = groupby_col == g
        indices = np.where(mask)[0]
        group_indices[g] = indices
    
    all_indices = np.concatenate([group_indices[g] for g in groups])
    all_indices = np.sort(all_indices)
    t_subset = time.time() - t
    
    # ========== LOAD X MATRIX ==========
    t = time.time()
    X_group = f["X"]
    n_vars = f["var"]["_index"].shape[0]
    
    # Load sparse matrix components
    X_data = np.array(X_group["data"][:])
    X_indices = np.array(X_group["indices"][:])
    X_indptr = np.array(X_group["indptr"][:])
    
    # Reconstruct sparse matrix and subset to dense
    X_csc = sparse.csc_matrix((X_data, X_indices, X_indptr), shape=(n_obs, n_vars))
    X_csr = X_csc.tocsr()
    X_subset = X_csr[all_indices, :].toarray().astype(np.float32)
    
    t_x = time.time() - t
    
    # ========== BUILD ANNDATA ==========
    t = time.time()
    obs_df = pd.DataFrame({groupby: groupby_col[all_indices]})
    var_index = np.array(f["var"]["_index"][:], dtype=object)
    var_df = pd.DataFrame(index=var_index)
    
    adata = ad.AnnData(
        X=X_subset,
        obs=obs_df.reset_index(drop=True),
        var=var_df
    )
    t_ad = time.time() - t
    
    # ========== NORMALIZE ==========
    t = time.time()
    sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)
    sc.pp.log1p(adata, base=2)
    t_norm = time.time() - t
    
    # ========== RANK GENES ==========
    t = time.time()
    sc.tl.rank_genes_groups(
        adata,
        groupby=groupby,
        method=method,
        groups=groups,
        n_genes=n_genes,
        use_raw=False
    )
    t_rank = time.time() - t
    
    # ========== EXTRACT RESULTS ==========
    result_df = sc.get.rank_genes_groups_df(adata, group=groups[0])
    
    t_total = time.time() - t_total_start
    
    # ========== PRINT SUMMARY ==========
    if verbose:
        print(f"\n⏱ TIMING:")
        print(f"  Zarr open:      {t_open:6.2f}s")
        print(f"  Obs groupby:    {t_obs:6.2f}s")
        print(f"  Find indices:   {t_subset:6.2f}s")
        print(f"  Load X (S3):    {t_x:6.2f}s ← Network bound")
        print(f"  Build AnnData:  {t_ad:6.2f}s")
        print(f"  Normalize:      {t_norm:6.2f}s")
        print(f"  Rank genes:     {t_rank:6.2f}s ({method})")
        print(f"  {'─'*40}")
        print(f"  TOTAL:          {t_total:6.2f}s")
        print(f"\n📊 RESULTS:")
        print(f"  Cells analyzed: {len(all_indices)}")
        print(f"  Genes tested:   {len(result_df)}")
        print(f"  Top gene:       {result_df.iloc[0]['names']} (score={result_df.iloc[0]['scores']:.2f})")
        print(f"{'='*70}\n")
    
    return result_df


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

if __name__ == "__main__":
    
    # Example 1: Allergy dataset
    print("█"*70)
    print("█ EXAMPLE 1: Allergy - Phase comparison")
    print("█"*70)
    
    result1 = rank_genes_zarr(
        s3_path="s3://scrnaseq-browser/corrected_doublet_decontX_mt_Duo_allergy.zarr",
        groupby="Phase",
        groups=["G1", "S"],
        method="t-test_overestim_var",
        n_genes=500
    )
    
    print("Top 10 ranked genes:")
    print(result1.head(10).to_string())
    print()
    
    # Example 2: EPDSC dataset
    print("\n" + "█"*70)
    print("█ EXAMPLE 2: EPDSC - Celltype comparison")
    print("█"*70)
    
    result2 = rank_genes_zarr(
        s3_path="s3://scrnaseq-browser/strict_epdsc_annotated_data_csc_full.zarr",
        groupby="celltype",
        groups=["DB_advanced", "DB_stress"],
        method="t-test_overestim_var",
        n_genes=500
    )
    
    print("Top 10 ranked genes:")
    print(result2.head(10).to_string())
    print()
    
    # Example 3: Save results
    # result1.to_csv("ranked_genes_allergy.csv", index=False)
    # result2.to_csv("ranked_genes_epdsc.csv", index=False)

██████████████████████████████████████████████████████████████████████
█ EXAMPLE 1: Allergy - Phase comparison
██████████████████████████████████████████████████████████████████████

🚀 Fast Zarr DGE: corrected_doublet_decontX_mt_Duo_allergy.zarr
   Phase: G1 vs S


/home/genos/gmunoz/SCRATCH/miniconda3/envs/sc_rna_env_python2/lib/python3.11/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)



⏱ TIMING:
  Zarr open:        0.31s
  Obs groupby:      0.09s
  Find indices:     0.00s
  Load X (S3):      2.44s ← Network bound
  Build AnnData:    0.17s
  Normalize:        0.18s
  Rank genes:       1.22s (t-test_overestim_var)
  ────────────────────────────────────────
  TOTAL:            4.41s

📊 RESULTS:
  Cells analyzed: 1641
  Genes tested:   500
  Top gene:       Malat1 (score=10.51)

Top 10 ranked genes:
     names     scores  logfoldchanges         pvals     pvals_adj
0   Malat1  10.511844        0.221443  6.243093e-25  4.495639e-21
1    Myh11   9.543515        2.220498  6.270770e-21  2.024518e-17
2      Des   9.242885        2.318835  9.554726e-20  2.804312e-16
3    Itga1   9.139962        2.019498  2.213800e-19  5.956045e-16
4      Fn1   8.802349        2.322964  4.221276e-18  9.085593e-15
5    Myom1   8.713914        2.009899  8.313304e-18  1.412605e-14
6     Tpm2   8.605091        2.010979  2.071370e-17  2.907573e-14
7     Mylk   8.485774        2.014569  5.516178e-17  

/home/genos/gmunoz/SCRATCH/miniconda3/envs/sc_rna_env_python2/lib/python3.11/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)



⏱ TIMING:
  Zarr open:        0.14s
  Obs groupby:      0.14s
  Find indices:     0.00s
  Load X (S3):      4.87s ← Network bound
  Build AnnData:    0.10s
  Normalize:        0.08s
  Rank genes:       0.57s (t-test_overestim_var)
  ────────────────────────────────────────
  TOTAL:            5.91s

📊 RESULTS:
  Cells analyzed: 1607
  Genes tested:   500
  Top gene:       PARD3 (score=54.98)

Top 10 ranked genes:
     names     scores  logfoldchanges          pvals      pvals_adj
0    PARD3  54.979454        7.946135  6.377614e-220  1.632350e-216
1  ANKRD11  53.438938        7.255250  1.792118e-228  5.504310e-225
2  SLCO3A1  53.024323        6.113606  9.808761e-267  1.506331e-262
3     SCEL  52.927246        6.628505  2.530687e-250  1.943188e-246
4    TENM4  51.096035        7.689261  7.374464e-207  9.437470e-204
5    TIAM1  50.555527        7.061035  6.422194e-215  9.862564e-212
6    MAML3  49.489681        7.494913  2.266973e-202  2.634141e-199
7  SLC24A3  49.377655        5.738601 